In [1]:
import tweepy
from config import bearer_token
import time
import pandas as pd
from tqdm import tqdm

In [2]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [2]:
result = []
user_dict = {}
query = '("type 1 diabetic" OR "type 1 diabetes" OR #T1D OR #T1DM OR #type1diabetes OR #t1diabetes OR #GBDoc OR #T1DLife OR #t1dlookslikeme) lang:en -is:retweet'
# Loop through each response object
for response in tqdm(tweepy.Paginator(client.search_all_tweets,
                                query = query,
                                user_fields = ['username', 'public_metrics', 'description', 'location'],
                                tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                expansions = 'author_id', # connet tweet data with user data
                                start_time = '2023-01-01T00:00:00Z',
                                end_time = '2023-01-31T00:00:00Z',
                                max_results = 500).flatten(limit= 10000)):
    time.sleep(1)

    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                            } 

    result.append({'id': response.data['id'],
                   'author_id': response.data['author_id'],
                   'created_at': response.data['created_at'],
                   'text': response.data['text'],
                   'retweets': response.data['public_metrics']['retweet_count'], 
                   'replies': response.data['public_metrics']['reply_count'],
                   'likes': response.data['public_metrics']['like_count'],
                   'quote_count': response.data['public_metrics']['quote_count']
                    })


df = pd.DataFrame(result)
df.to_csv('Twitter_T1DM_202301.csv')

